<a href="https://colab.research.google.com/github/vash02/research-rep-agent/blob/main/agent_iterative.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers accelerate

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "deepseek-ai/deepseek-coder-6.7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 129.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvji

tokenizer_config.json:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [2]:
!pip install inspect-ai

INFO: pip is looking at multiple versions of s3fs to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 91.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.2/119.2 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 685.6/685.6 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 123.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of 

In [3]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 61.0 MB/s eta 0:00:00


utils

In [4]:
from typing import List, cast
import tiktoken
from inspect_ai.model._chat_message import (
    ChatMessage,
    ChatMessageAssistant,
    ChatMessageSystem,
    ChatMessageTool,
    ChatMessageUser,
)

def handle_message_len(
    msg: ChatMessage,
    tokenizer,
    max_tokens: int,
) -> ChatMessage:
    def truncate_string(text: str, tokens: list[int], max_tokens: int) -> str:
        if len(tokens) > max_tokens:
            keep = max_tokens // 2
            first = tokenizer.decode(tokens[:keep])
            last = tokenizer.decode(tokens[-keep:])
            return first + "\n...[truncated]...\n" + last
        return text

    if isinstance(msg.content, str):
        tokens = tokenizer.encode(msg.content, disallowed_special=())
        msg.content = truncate_string(msg.content, tokens, max_tokens)
    return msg

def prune_messages(
    messages: List[ChatMessage], prune_individual: bool = True, max_context_tokens: int = 25000
) -> List[ChatMessage]:
    """Prune and truncate messages to prevent OOM and keep system prompt."""
    system_msgs = [m for m in messages if m.role == "system"]
    conversation = [m for m in messages if m.role != "system"]

    # Always keep the first user message (typically the task instruction)
    first_user = next((m for m in conversation if m.role == "user"), None)

    # Remove oldest ~50% of conversation (keep newer half)
    start = max(1, len(conversation) // 2)
    preserved = [first_user] if first_user else []
    preserved.extend(conversation[start:])

    # Tool-message consistency
    valid_msgs = []
    active_tool_ids = set()

    for m in preserved:
        if isinstance(m, ChatMessageAssistant):
            active_tool_ids = {tc.id for tc in (m.tool_calls or [])}
            valid_msgs.append(m)
        elif isinstance(m, ChatMessageTool) and getattr(m, "tool_call_id", None) in active_tool_ids:
            valid_msgs.append(m)
        elif isinstance(m, ChatMessageUser):
            active_tool_ids.clear()
            valid_msgs.append(m)

    if prune_individual:
        tokenizer = tiktoken.get_encoding("cl100k_base")
        MAX_PER_MSG = max_context_tokens // max(len(valid_msgs), 1)
        valid_msgs = [handle_message_len(m, tokenizer, MAX_PER_MSG) for m in valid_msgs]

    return cast(List[ChatMessage], system_msgs + valid_msgs)


In [5]:
import torch
from inspect_ai.model import ModelOutput
from inspect_ai.model._chat_message import ChatMessageAssistant
from inspect_ai.model._model_output import ChatCompletionChoice

class LocalHFModelWrapper:
    def __init__(self, model, tokenizer, max_new_tokens=512):
        self.name = "local-llama3"
        self.model = model
        self.tokenizer = tokenizer
        self.max_new_tokens = max_new_tokens
        self.total_retry_time = 0
        self.api = None  # leave this None if not integrating with registry

    async def generate(self, input, tools=None, tool_choice=None, config=None):
        # Step 1: Construct prompt
        prompt = ""
        for msg in input:
            if msg.role == "system":
                prompt += f"[System]\n{msg.content}\n\n"
            elif msg.role == "user":
                prompt += f"[User]\n{msg.content}\n\n"
            elif msg.role == "assistant":
                prompt += f"[Assistant]\n{msg.content}\n\n"

        # Step 2: Tokenize safely and truncate if needed
        inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True)
        inputs = {k: v.to(self.model.device) for k, v in inputs.items()}

        # Step 3: Generate with model
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=self.max_new_tokens,
                pad_token_id=self.tokenizer.eos_token_id,
            )

        # Step 4: Decode response safely
        decoded = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Optional: Strip original prompt to get just the response (optional, depends on tokenizer)
        response = decoded[len(prompt):].strip()

        # Step 5: Return structured output
        return ModelOutput(
            model=self.name,
            choices=[ChatCompletionChoice(message=ChatMessageAssistant(content=response))]
        )


In [6]:
hf_model = LocalHFModelWrapper(model=model, tokenizer=tokenizer)

In [7]:
import json
from pathlib import Path
from inspect_ai.model._chat_message import ChatMessage, ChatMessageUser, ChatMessageAssistant, ChatMessageSystem


class DiskBackedMessageHistory:
    def __init__(self, path="messages.jsonl", memory_window=3):
        self.path = Path(path)
        self.memory_window = memory_window
        self.messages_in_memory = []

        # Initialize file
        if self.path.exists():
            self.messages_in_memory = list(self._load_last_n(memory_window))
        else:
            self.path.write_text("")

    def append(self, message):
        self.messages_in_memory.append(message)
        with self.path.open("a") as f:
            f.write(json.dumps(message.model_dump()) + "\n")

        # Keep only last N messages in memory
        if len(self.messages_in_memory) > self.memory_window:
            self.messages_in_memory.pop(0)

    def get_recent(self):
        return self.messages_in_memory

    def __len__(self):
      with self.path.open() as f:
          return sum(1 for _ in f)

    def _deserialize_chat_message(self, raw: dict):
        role = raw.get("role")
        if role == "user":
            return ChatMessageUser(**raw)
        elif role == "assistant":
            return ChatMessageAssistant(**raw)
        elif role == "system":
            return ChatMessageSystem(**raw)
        else:
            raise ValueError(f"Unsupported role: {role}")

    def _load_last_n(self, n):
      with self.path.open() as f:
          lines = f.readlines()[-n:]
      return [self._deserialize_chat_message(json.loads(line)) for line in lines]



In [8]:
import asyncio
import json
import re
from pathlib import Path
from types import SimpleNamespace
from inspect_ai.model._chat_message import ChatMessageUser, ChatMessageAssistant
from inspect_ai.model._model_output import ChatCompletionChoice
import gc


def extract_code_blocks(text, lang="python"):
    pattern = rf"```{lang}\n(.*?)\n```"
    return re.findall(pattern, text, re.DOTALL)


async def agent_loop(state, model, max_messages=50):
    step = 0

    while not state.completed:
        if len(state.messages) >= max_messages:
            print("Reached max message limit.")
            state.completed = True
            break

        # Reminder every few steps
        if step > 0 and step % 5 == 0:
            state.messages.append(ChatMessageUser(content="Reminder: Keep solving the task."))

        # Prune message history (before generate)
        pruned_messages = prune_messages(state.messages, prune_individual=True)

        # Limit to last 20 messages for context
        input_for_model = pruned_messages[-20:]

        output = await model.generate(input=input_for_model)
        assistant_msg = output.choices[0].message
        state.messages.append(assistant_msg)

        # Free GPU RAM
        torch.cuda.empty_cache()
        gc.collect()

        # Extract and print code
        code_blocks = extract_code_blocks(assistant_msg.content, lang="python")
        for code in code_blocks:
            print(f"\n🧠 Step {step} - Generated Code Block:\n{code}\n")
            with open("generated_code_log.py", "a") as f:
                f.write(f"# === Step {step} ===\n{code}\n\n")

        step += 1

    return state



In [9]:
from inspect_ai.model._chat_message import ChatMessageUser
from pathlib import Path

async def run():
    with open("/content/drive/MyDrive/home/instructions.txt") as f:
        instructions = f.read()

    state = SimpleNamespace(
        messages=[ChatMessageSystem(content=instructions)],
        completed=False
    )

    await agent_loop(state, model=hf_model)  # ensure hf_model is loaded and defined

# In notebook
await run()


Reached max message limit.
